In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [20]:
def load_and_process_lithium_data():
    # File Paths (Ensure these match your actual file names)
    files = {
        'imp_hist': 'Graphite import Forecsting.xlsx',
        'imp_2025': 'Graphite import 2025.xlsx',
        'exp_hist': 'Graphite export Forecasting.xlsx',
        'exp_2025': 'Graphite export 2025.xlsx'
    }

    def read_clean(f_hist, f_curr):
        # Load History and Current(Forecast) files
        dh = pd.read_excel(f_hist)
        dc = pd.read_excel(f_curr)

        # Clean columns (Remove 'Total' column if it exists, strictly keep months)
        # We assume structure: Country, Month-Year...
        if 'Total' in dh.columns: dh = dh.drop(columns=['Total'])
        if 'Total' in dc.columns: dc = dc.drop(columns=['Total'])

        # Melt to Long Format
        dh_long = dh.melt(id_vars='Country/Region', var_name='Date', value_name='Value')
        dc_long = dc.melt(id_vars='Country/Region', var_name='Date', value_name='Value')

        # Combine
        full = pd.concat([dh_long, dc_long])
        full['Date'] = pd.to_datetime(full['Date'], format='%b-%Y')
        return full.sort_values('Date')

    # Load All
    df_imp = read_clean(files['imp_hist'], files['imp_2025'])
    df_exp = read_clean(files['exp_hist'], files['exp_2025'])

    # --- PREPARE DATA FOR VIEW 1: STRATEGIC GAP (Time Series) ---
    # Filter for 'Total' rows
    imp_total = df_imp[df_imp['Country/Region'] == 'Total'].set_index('Date')['Value']
    exp_total = df_exp[df_exp['Country/Region'] == 'Total'].set_index('Date')['Value']

    gap_df = pd.DataFrame({'Import': imp_total, 'Export': exp_total}).dropna()
    gap_df['Gap'] = gap_df['Import'] - gap_df['Export']  # Positive = Deficit
    gap_df = gap_df.reset_index()

    # --- PREPARE DATA FOR VIEW 2: GEOPOLITICAL RISK (Dependency) ---
    # Filter for 2025 (Forecast Period) and specific countries
    df_2025 = df_imp[(df_imp['Date'].dt.year == 2025) & (df_imp['Country/Region'] != 'Total')]

    # Aggregate total import by country for 2025
    dep_df = df_2025.groupby('Country/Region')['Value'].sum().reset_index()
    total_val = dep_df['Value'].sum()

    # Calculate Dependency Ratio (%)
    dep_df['Ratio'] = (dep_df['Value'] / total_val) * 100
    dep_df = dep_df.sort_values('Ratio', ascending=False)

    return gap_df, dep_df

In [21]:
gap_data, risk_data = load_and_process_lithium_data()

In [14]:
# Create a Figure with a secondary axis-like layout using domain
fig = go.Figure()

# --- TRACES FOR VIEW 1: STRATEGIC GAP ---
# Trace 0: Import Line
fig.add_trace(go.Scatter(
    x=gap_data['Date'], y=gap_data['Import'],
    name='Demand (Import)', line=dict(color='blue', width=3),
    visible=True
))

# Trace 1: Export Line
fig.add_trace(go.Scatter(
    x=gap_data['Date'], y=gap_data['Export'],
    name='Supply (Export)', line=dict(color='green', width=3),
    visible=True
))

# Trace 2: Gap Fill (Visualizing the Deficit)
fig.add_trace(go.Scatter(
    x=gap_data['Date'], y=gap_data['Import'],
    fill='tonexty', fillcolor='rgba(255, 0, 0, 0.1)',
    name='Trade Gap',
    line=dict(width=0),
    visible=True
))

# --- TRACES FOR VIEW 2: GEOPOLITICAL RISK ---
# Trace 3: Pie Chart (Dependency Ratio)
fig.add_trace(go.Pie(
    labels=risk_data['Country/Region'].head(5),
    values=risk_data['Value'].head(5),
    name='Dependency Ratio',
    hole=0.4,
    domain={'x': [0, 0.5], 'y': [0.2, 0.8]}, # Left side
    visible=False
))

# Trace 4: Table (Detailed Values)
fig.add_trace(go.Table(
    header=dict(values=['Country', 'Total Import ($M)', 'Dependency Ratio (%)'],
                fill_color='paleturquoise', align='left'),
    cells=dict(values=[risk_data['Country/Region'],
                       risk_data['Value'].round(2),
                       risk_data['Ratio'].map('{:.1f}%'.format)],
               fill_color='lavender', align='left'),
    domain={'x': [0.55, 1], 'y': [0, 1]}, # Right side
    visible=False
))

In [5]:
fig.update_layout(
    title="Lithium Strategic Supply Dashboard (MVP)",
    template="plotly_white",
    height=600,

    # The Dropdown Menu
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            x=0.5, y=1.15,
            showactive=True,
            buttons=[
                # BUTTON 1: Show Trend (Gap Analysis)
                dict(
                    label="Strategic Gap View",
                    method="update",
                    args=[
                        {"visible": [True, True, True, False, False]}, # Show Traces 0,1,2 | Hide 3,4
                        {"title": "Lithium Supply-Demand Forecast (2017-2025)",
                         "xaxis.visible": True, "yaxis.visible": True}
                    ]
                ),
                # BUTTON 2: Show Risk (Dependency)
                dict(
                    label="Geopolitical Risk View",
                    method="update",
                    args=[
                        {"visible": [False, False, False, True, True]}, # Hide Traces 0,1,2 | Show 3,4
                        {"title": "Import Dependency Ratio & Country Risk (2025)",
                         "xaxis.visible": False, "yaxis.visible": False} # Hide axes for Pie/Table
                    ]
                ),
            ]
        )
    ]
)

fig.show()

In [22]:
fig.update_layout(
    title="Graphite Strategic Supply Dashboard (MVP)",
    template="plotly_white",
    height=600,

    # The Dropdown Menu
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            x=0.5, y=1.15,
            showactive=True,
            buttons=[
                # BUTTON 1: Show Trend (Gap Analysis)
                dict(
                    label="Strategic Gap View",
                    method="update",
                    args=[
                        {"visible": [True, True, True, False, False]}, # Show Traces 0,1,2 | Hide 3,4
                        {"title": "Graphite Supply-Demand Forecast (2017-2025)",
                         "xaxis.visible": True, "yaxis.visible": True}
                    ]
                ),
                # BUTTON 2: Show Risk (Dependency)
                dict(
                    label="Geopolitical Risk View",
                    method="update",
                    args=[
                        {"visible": [False, False, False, True, True]}, # Hide Traces 0,1,2 | Show 3,4
                        {"title": "Import Dependency Ratio & Country Risk (2025)",
                         "xaxis.visible": False, "yaxis.visible": False} # Hide axes for Pie/Table
                    ]
                ),
            ]
        )
    ]
)

fig.show()

In [15]:
fig.update_layout(
    title="Copper Strategic Supply Dashboard (MVP)",
    template="plotly_white",
    height=600,

    # The Dropdown Menu
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            x=0.5, y=1.15,
            showactive=True,
            buttons=[
                # BUTTON 1: Show Trend (Gap Analysis)
                dict(
                    label="Strategic Gap View",
                    method="update",
                    args=[
                        {"visible": [True, True, True, False, False]}, # Show Traces 0,1,2 | Hide 3,4
                        {"title": "Lithium Supply-Demand Forecast (2017-2025)",
                         "xaxis.visible": True, "yaxis.visible": True}
                    ]
                ),
                # BUTTON 2: Show Risk (Dependency)
                dict(
                    label="Geopolitical Risk View",
                    method="update",
                    args=[
                        {"visible": [False, False, False, True, True]}, # Hide Traces 0,1,2 | Show 3,4
                        {"title": "Import Dependency Ratio & Country Risk (2025)",
                         "xaxis.visible": False, "yaxis.visible": False} # Hide axes for Pie/Table
                    ]
                ),
            ]
        )
    ]
)

fig.show()